In [ ]:
#default_exp query

In [ ]:
#export
import requests
from pathlib import Path

In [ ]:
#export
MOSAICO = 'http://sistemas.anatel.gov.br/se/public/file/b/srd/estacao_rd.zip'
RADCOM = """
use SITARWEB

select f.MedFrequenciaInicial as 'Frequência', 
       uf.SiglaUnidadeFrequencia as 'Unidade',
       es.MedLatitudeDecimal as 'Latitude' ,
	   es.MedLongitudeDecimal as 'Longitude',
       Sitarweb.dbo.FN_SRD_RetornaIndFase(PB.NumServico, Pr.idtPedidoRadcom) as 'Fase',
       Sitarweb.dbo.FN_SRD_RetornaSiglaSituacao(h.IdtHabilitacao, Es.IdtEstacao) as 'Situação',
       es.NumEstacao as 'Numero da Estação',
	   e.NumCnpjCpf as 'CNPJ',
	   e.NomeEntidade as 'Entidade',
	   m.NomeMunicipio as 'Município',
	   pb.SiglaUF as 'UF'	 
from ENTIDADE e
inner join HABILITACAO h on h.IdtEntidade = e.IdtEntidade
inner join SRD_PEDIDORADCOM pr on pr.IdtHabilitacao = h.IdtHabilitacao
inner join SRD_PLANOBASICO pb on pb.IdtPlanoBasico = pr.IdtPlanoBasico
inner join estacao es on es.IdtHabilitacao = h.IdtHabilitacao
inner join FREQUENCIA f on f.IdtEstacao = es.IdtEstacao
inner join UnidadeFrequencia uf on uf.IdtUnidadeFrequencia = f.IdtUnidadeFrequencia
inner join Municipio m on m.CodMunicipio = pb.CodMunicipio
where h.NumServico = '231'
"""

In [ ]:
STEL = """
IF OBJECT_ID('tempDB..##faixas','U') is not null 
drop table ##faixas 
create table ##faixas (id int not null, faixa varchar(20), inic float, fim float,); 
insert into ##faixas values(0,'De 20 MHz - 6 GHz','20000', '6000000'); 

select distinct 
f.MedTransmissaoInicial as 'Frequência',
uf.SiglaUnidadeFrequencia as 'Unidade',
e.MedLatitudeDecimal as 'Latitude',
e.MedLongitudeDecimal as 'Longitude',
e.NumServico as 'Número do Serviço',
e.NumEstacao as 'Número da estação',
ent.NomeEntidade as 'Entidade',
mu.NomeMunicipio as 'Município', 
e.SiglaUf as 'UF',
ent.NumCnpjCpf as 'CNPJ',
h.NumFistel as 'Fistel'
from contrato c
inner join estacao e on e.IdtContrato = c.Idtcontrato
inner join frequencia f on f.IdtEstacao = e.IdtEstacao
inner join HABILITACAO h on h.IdtHabilitacao = c.IdtHabilitacao 
inner join entidade ent on ent.IdtEntidade = h.IdtEntidade 
inner join endereco en on en.IdtEstacao = e.IdtEstacao 
inner join Municipio mu on mu.CodMunicipio = en.CodMunicipio 
inner join Servico s on s.NumServico = h.NumServico and s.IdtServicoAreaAtendimento = 4
left join UnidadeFrequencia uf on uf.IdtUnidadeFrequencia = f.IdtUnidadeTransmissao
left outer join ##faixas fx on (
	(fx.inic <= f.MedRecepcaoInicialKHz and fx.fim >= f.MedRecepcaoInicialKHz) 
	or (fx.inic <= f.MedTransmissaoInicialKHz and fx.fim >= f.medtransmissaoinicialkhz)
	or (fx.inic <= f.MedFrequenciaInicialKHz and fx.fim >= f.MedFrequenciaInicialKHz)
	or (fx.inic <= f.MedFrequenciaFinalKHz and fx.fim >= f.MedFrequenciaFinalKHz)
	) 
where e.DataExclusao is null and 
fx.faixa is not null and
f.MedTransmissaoInicial is not null
and h.NumServico <> '010'
"""

In [ ]:
#export
def download_mosaico():
    file = requests.get(MOSAICO)
    with open(Path.cwd() / 'files' / 'mosaico.zip', 'wb') as mosaico:
        mosaico.write(file.content)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
Converted process.ipynb.
Converted queries.ipynb.
